## 지정한 구청의 동별 세대수 가져오기
+ 작성: 임경호

### 1. 지정한 년월에 사용 가능한 최신의 법정동, 행정동 코드 자료를 테이블에서 가져온다.

In [1]:
import pandas as pd
import requests
from tqdm.notebook import tqdm
import json
from datetime import datetime
import calendar

import sys
module_path = "D:\PythonProject\data-gatherer\common"
sys.path.append(module_path)
import dbconnect

In [3]:
region = "대구광역시"
#추출하고자 하는 년월
input_year = 2023
input_month = 8
# 년월: yyyymm 형식 문자열
month = str(input_year) + str(input_month).zfill(2)   
last_day = calendar.monthrange(input_year, input_month)[1]
# print(year, month, last_day)
month_end_date = datetime(input_year, input_month, last_day).strftime("%Y%m%d")

In [4]:
# 해당 년월에 사용가능한, 저장된 데이터의 마지막 일자 가져오기
conn = dbconnect.db_connect("DEMO_DM")
cur = conn.cursor()

query = "SELECT 업데이트일자 FROM city WHERE 업데이트일자 <= '" + month_end_date + "' and 시도명 = '" + region + "' ORDER BY 업데이트일자 DESC LIMIT 1"
cur.execute(query)
row = cur.fetchone()
stored_date = row[0]     

conn.close()

In [6]:
# 지역코드 테이블에서 데이터 읽어오기
engine = dbconnect.db_engine("DEMO_DM")
query = "SELECT * FROM city WHERE 업데이트일자 = '" + stored_date + "' and 시도명 = '" + region + "'"
result = pd.read_sql(query, engine)

In [7]:
df_code_table = result.copy()
df_code_table

,업데이트일자,행정동코드,시도명,시군구명,읍면동명,법정동코드,동리명
0,20230703,2700000000,대구광역시,,,2700000000,대구광역시
1,20230703,2711000000,대구광역시,중구,,2711000000,중구
2,20230703,2711051700,대구광역시,중구,동인동,2711010100,동인동1가
3,20230703,2711051700,대구광역시,중구,동인동,2711010200,동인동2가
4,20230703,2711051700,대구광역시,중구,동인동,2711010300,동인동3가
...,...,...,...,...,...,...,...
500,20230703,2772037000,대구광역시,군위군,삼국유사면,2772037027,양지리
501,20230703,2772037000,대구광역시,군위군,삼국유사면,2772037028,낙전리
502,20230703,2772037000,대구광역시,군위군,삼국유사면,2772037029,가암리
503,20230703,2772037000,대구광역시,군위군,삼국유사면,2772037030,석산리


In [8]:
# 전제 코드 데이터
df_code_table_pre = df_code_table.copy()
df_code_table_pre.shape

(505, 7)

In [9]:
# '시군구' 단위 및 '읍면동' 단위의 데이터만 남겨놓는다
df_code_table_pre = df_code_table_pre[df_code_table_pre['법정동코드'].str.endswith('00')]
# '시군구' 단위 데이터 삭제
idx = df_code_table_pre[df_code_table_pre['법정동코드'].str.endswith('00000')].index
df_code_table_pre.drop(idx, inplace=True)
# 최종적으로 '읍면동' 코드만 남는다.
df_code_table_pre.shape

(304, 7)

In [10]:
#구 별로 법정동코드를 추출하여 list형식으로 변환
def make_code_list_for_daegu(df,gu_name):
    df_gu= df[df['시군구명']==gu_name]
    list_gu=  df_gu['법정동코드']
    list_gu = list_gu.unique()
    return list_gu

In [11]:
# 각 도시별 법정동 코드 불러오기

#대구광역시
df_daegu_code = df_code_table_pre[df_code_table_pre['시도명'].str.contains('대구광역시')]
df_daegu_code= df_daegu_code.dropna(subset=['읍면동명']) #null값 제거
idx = df_daegu_code[df_daegu_code['법정동코드'].str[-4:] == "0000"].index #리 값 제거
df_daegu_code.drop(idx , inplace=True)

list_daegu_cd =  df_daegu_code['법정동코드']
list_daegu_cd = list_daegu_cd.unique() #대구광역시의 법정동코드

#중간에 오류가 날 수 있어 구별로 나누어 작업 진행
dict_region_codes = {}  # 딕셔너리
dict_region_codes["남구"] =  make_code_list_for_daegu(df_daegu_code,"남구")
dict_region_codes["달서구"] =  make_code_list_for_daegu(df_daegu_code,"달서구")
dict_region_codes["동구"] =  make_code_list_for_daegu(df_daegu_code,"동구")
dict_region_codes["북구"] =  make_code_list_for_daegu(df_daegu_code,"북구")
dict_region_codes["서구"] =  make_code_list_for_daegu(df_daegu_code,"서구")
dict_region_codes["수성구"] =  make_code_list_for_daegu(df_daegu_code,"수성구")
dict_region_codes["중구"] =  make_code_list_for_daegu(df_daegu_code,"중구")

In [12]:
# 각 지역별로 지역코드(동별) 개수 확인
for key in dict_region_codes:
    print(f'{key}\t{len(dict_region_codes[key])}')

남구	3
달서구	24
동구	45
북구	31
서구	9
수성구	26
중구	57


### 3. Open API를 이용한 세대수 데이터 추출

In [13]:
import warnings
from time import sleep
warnings.filterwarnings(action='ignore')

service_url = 'http://apis.data.go.kr/1741000/stdgPpltnHhStus'
# 일반 인증키(Encoding)	
api_key = 'IVgu%2FZBjA6hpLryyEOpySC2RhogOhaJIUqlXN8Uyj3Gxw4s3dX0qMxfgXMTLl60%2Fs2EYAMUsyyzTqwVOnjoIhg%3D%3D'
level = '4'     # 조회결과구분. 광역시도 단위 : 1, 시군구 단위 : 2, 읍면동 단위 : 3, 읍면동 통반 단위 : 4(기본값 : 4)
reg_code = '1'  # 등록구분. 전체:1, 거주자:2, 거주불명자:3, 재외국민:4(기본값 : 1)
req_type = 'json'   # 타입. XML, JSON(기본값 : XML)
pageRows = '8000'    # 페이지 크기. 페이지당 목록 수(1~100)(기본값 : 10)
pageNo = '1'        # 페이지 번호. 기본값 : 1

In [14]:
#시군구별 세대수를 알아보는 함수 >> 시군구 자료와 데이터 비교
def know_hhcnt(df):
    df['hhCnt'] =df['hhCnt'].astype(str).astype(int)
    df['femlNmprCnt'] =df['femlNmprCnt'].astype(str).astype(int)
    df['totNmprCnt'] =df['totNmprCnt'].astype(str).astype(int)
    df['maleNmprCnt'] =df['maleNmprCnt'].astype(str).astype(int)
    know_hhcnt = df.groupby('sggNm').sum()
    return know_hhcnt

#기존의 표와 같이 보기 편하게 순서 변경 및 컬럼 변경
def name_sort_change(df):
    #이름 변경
    df.rename(columns={'hhCnt':'세대수','tong':'통','femlNmprCnt':'여자인구수','stdgCd':'법정동코드','maleFemlRate':'남여비율',
                       'stdgNm':'법정동명','ban':'반','totNmprCnt':'총인구수','ctpvNm':'시도명','maleNmprCnt':'남자인구수',
                       'sggNm':'시군구명','dongNm':'행정동명','hhNmpr':'세대당인구','admmCd':'행정기관코드',
                       'statsYm':'통계년월'},inplace=True)
    #기존의 표와 같이 순서 변경
    df = df[['통계년월','법정동코드','시도명','시군구명','법정동명','행정기관코드','행정동명','통',
             '반','총인구수','세대수','세대당인구','남자인구수','여자인구수','남여비율']]
    df['업데이트일자'] = datetime.now().strftime("%Y%m%d")
    return df

In [15]:
# 각 지역별로 데이터 수집 결과 초기화
dict_region_data = {}
dict_region_results = {}
for key in dict_region_codes:
    if key in ['중구']:
        dict_region_results[key] = 0
print(dict_region_results)

{'중구': 0}


In [16]:
df_region_cd_list = dict_region_codes['중구']

In [18]:
df_region_cd_list

array(['2711010100', '2711010200', '2711010300', '2711010400',
       '2711010500', '2711010600', '2711010700', '2711010800',
       '2711011200', '2711011500', '2711011700', '2711011900',
       '2711012000', '2711012100', '2711012200', '2711012300',
       '2711012400', '2711012500', '2711012600', '2711012700',
       '2711012800', '2711014300', '2711014800', '2711015000',
       '2711015100', '2711010900', '2711011000', '2711011100',
       '2711011300', '2711011400', '2711011600', '2711011800',
       '2711013000', '2711013100', '2711013400', '2711013600',
       '2711013700', '2711013800', '2711013900', '2711014000',
       '2711014200', '2711014400', '2711014500', '2711014600',
       '2711014700', '2711014900', '2711015200', '2711015300',
       '2711012900', '2711013200', '2711013300', '2711013500',
       '2711014100', '2711015500', '2711015400', '2711015600',
       '2711015700'], dtype=object)

In [19]:
# 각 지역별로 API 호출하여 동별 세대수 데이터 가져오기
df_region_data = pd.DataFrame()
for i in df_region_cd_list: 
    # 최대 5회 시도
    num_retry = 5
    for retry in range(num_retry):
        sleep(5)
        try:        
            print(i, " try ", retry, end=" ")        
            url = f'{service_url}/selectStdgPpltnHhStus?serviceKey={api_key}&stdgCd={i}&srchFrYm={month}&srchToYm={month}&lv={level}&regSeCd={reg_code}&type={req_type}&numOfRows={pageRows}&pageNo={pageNo}'
            # url = 'https://apis.data.go.kr/1741000/stdgPpltnHhStus/selectStdgPpltnHhStus?serviceKey=IVgu%2FZBjA6hpLryyEOpySC2RhogOhaJIUqlXN8Uyj3Gxw4s3dX0qMxfgXMTLl60%2Fs2EYAMUsyyzTqwVOnjoIhg%3D%3D&stdgCd='+i+'&srchFrYm='+month+'&srchToYm='+month+'&lv=4&regSeCd=1&type=json&numOfRows=8000&pageNo=1'
            response = requests.get(url,verify=False)
            #데이터 값 출력해보기
            contents = response.text
            json_ob = json.loads(contents)
            body = json_ob['Response']['items']['item']
            dataframe = pd.json_normalize(body)
            df_region_data = pd.concat([df_region_data, dataframe],axis = 0) #데이터 결합
            print(i, " ...done")
            break
        except Exception:
            continue            
    sleep(5)

2711010100  try  0 2711010100  ...done
2711010200  try  0 2711010200  ...done
2711010300  try  0 2711010300  ...done
2711010400  try  0 2711010400  ...done
2711010500  try  0 2711010500  ...done
2711010600  try  0 2711010600  ...done
2711010700  try  0 2711010700  ...done
2711010800  try  0 2711010800  ...done
2711011200  try  0 2711011200  ...done
2711011500  try  0 2711011500  ...done
2711011700  try  0 2711011700  ...done
2711011900  try  0 2711011900  ...done
2711012000  try  0 2711012000  ...done
2711012100  try  0 2711012100  ...done
2711012200  try  0 2711012200  ...done
2711012300  try  0 2711012300  ...done
2711012400  try  0 2711012400  ...done
2711012500  try  0 2711012500  try  1 2711012500  try  2 2711012500  ...done
2711012600  try  0 2711012600  ...done
2711012700  try  0 2711012700  ...done
2711012800  try  0 2711012800  ...done
2711014300  try  0 2711014300  ...done
2711014800  try  0 2711014800  ...done
2711015000  try  0 2711015000  ...done
2711015100  try  0 2711015

In [22]:
df_region_data

,hhCnt,tong,femlNmprCnt,stdgCd,maleFemlRate,stdgNm,ban,totNmprCnt,ctpvNm,maleNmprCnt,sggNm,dongNm,hhNmpr,admmCd,statsYm
0,120,1,163,2711010100,0.96,동인동1가,1,319,대구광역시,156,중구,동인동,2.66,2711051700,202308
1,9,1,7,2711010100,0.86,동인동1가,2,13,대구광역시,6,중구,동인동,1.44,2711051700,202308
2,1,1,1,2711010100,0.00,동인동1가,4,1,대구광역시,0,중구,동인동,1.00,2711051700,202308
3,2,1,0,2711010100,2.00,동인동1가,5,2,대구광역시,2,중구,동인동,1.00,2711051700,202308
4,1,1,1,2711010100,0.00,동인동1가,6,1,대구광역시,0,중구,동인동,1.00,2711051700,202308
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,41,18,55,2711015700,0.91,대봉동,4,105,대구광역시,50,중구,대봉1동,2.56,2711068000,202308
163,32,18,44,2711015700,0.91,대봉동,5,84,대구광역시,40,중구,대봉1동,2.63,2711068000,202308
164,38,18,52,2711015700,0.58,대봉동,6,82,대구광역시,30,중구,대봉1동,2.16,2711068000,202308
165,59,18,80,2711015700,0.75,대봉동,7,140,대구광역시,60,중구,대봉1동,2.37,2711068000,202308


In [23]:
#데이터 컬럼명 변경 및 위치 변경
df_region_data = name_sort_change(df_region_data)

In [24]:
df_region_data.head()

,통계년월,법정동코드,시도명,시군구명,법정동명,행정기관코드,행정동명,통,반,총인구수,세대수,세대당인구,남자인구수,여자인구수,남여비율,업데이트일자
0,202308,2711010100,대구광역시,중구,동인동1가,2711051700,동인동,1,1,319,120,2.66,156,163,0.96,20230908
1,202308,2711010100,대구광역시,중구,동인동1가,2711051700,동인동,1,2,13,9,1.44,6,7,0.86,20230908
2,202308,2711010100,대구광역시,중구,동인동1가,2711051700,동인동,1,4,1,1,1.00,0,1,0.00,20230908
3,202308,2711010100,대구광역시,중구,동인동1가,2711051700,동인동,1,5,2,2,1.00,2,0,2.00,20230908
4,202308,2711010100,대구광역시,중구,동인동1가,2711051700,동인동,1,6,1,1,1.00,0,1,0.00,20230908


# 5. 법정동, 행정동 세대수 데이터  SQL에 삽입 

In [25]:
conn = dbconnect.db_connect("DEMO_DW")
cursor = conn.cursor()

query = f'SELECT EXISTS (SELECT * FROM household_dong WHERE 통계년월 = {month})'

cursor.execute(query)
row = cursor.fetchone()
data_exist = row[0]     # 저장된 데이터의 유무(1 - 데이터 있음)

conn.close()

In [26]:
conn = dbconnect.db_connect("DEMO_DW")
cur = conn.cursor()

for row in df_region_data.itertuples():
    sql = "insert into household_dong (통계년월, 법정동코드, 시도명, 시군구명, 법정동명, \
                                        행정기관코드, 행정동명, 통, 반, 총인구수, 세대수, 세대당인구, \
                                        남자인구수, 여자인구수, 남여비율, 업데이트일자) \
            values (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    cur.execute(sql, (row[1], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[9], row[10], row[11], row[12], row[13], row[14], row[15], row[16]))

conn.commit()
print(f'{month} 데이터를 저장하였습니다.')
conn.close()

202308 데이터를 저장하였습니다.
